In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import operator
import collections
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score,precision_score,f1_score
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
from scipy import stats
from sklearn.tree import DecisionTreeRegressor
from sklearn import preprocessing
import types

# Reading dataset :  1000 x 81  
- Columns having NULL values > 70% serve no purpose.
There are 4 such columns which are dropped alog with the first column named 'Id'.  
- Replacing NA values in numerical data with the mean & in that of categorical data with the mode.  

# Node Decision
- For deciding which attribute needs to become the node at each level, we are calculating the **MSE** of the columns of dataset and choosing the minimum val and the corresponding attribute for our decision.
- MSE calculation involves taking the unique values of the column.
- If the column is categorical, then each unique value is considered for node value and mse calcualtion
- If numerical, we take mean of adjacent unique values for mse calculation.

# Splitting Decision

- Based on the node value selected, the split decision needs to be taken.
- For numerical attribute, rows whose values are <= node value are considered for left subtree creation and remaining for right subtree.
- For categorical attribute, rows whose values are == node value are considered for left subtreeand remaining for right subtree.


In [0]:
class Node:

  def __init__(self,at,val):
    self.left=None
    self.right=None
    self.attribute=at
    self.value=val
    # self.leaf=False


In [0]:
class DecisionTree:

  def __init_(self):
    self.columnx=[-1]


  def train(self,path):
    self.columnx=[-1]
    self.dataset=pd.read_csv(path)
    # print(dataset)
    self.dataset=self.dataset.drop(self.dataset.columns[0],axis=1)
    # print(dt)
    # print(self.dataset.nunique())
    self.dataset['MSZoning']=self.dataset['MSZoning'].replace('C (all)','C')
    self.dataset['MasVnrType']=self.dataset['MasVnrType'].replace('None',np.nan)
    # print(dataset[dataset.columns[1]])
    nullvaluecount=self.dataset.isnull().sum(axis=0)
    # print(type(nullvaluecount))
    self.dropcolumn=[]
    for i in range(len(nullvaluecount)):
      if(nullvaluecount[i]/1000>=0.7):
        self.dropcolumn.append(i)
    self.dataset=self.dataset.drop(self.dataset.columns[self.dropcolumn],axis=1)
    # print(dataset)
    # print('Mean')
    self.dataset.fillna(self.dataset.mean(),inplace=True)
    # print(dataset)
    nullvaluecount=self.dataset.isnull().sum(axis=0)
    # print(nullvaluecount)
    self.dataset.fillna(self.dataset.mode().iloc[0],inplace=True)
    # self.datanumpy=self.dataset.to_numpy()
    # print('Mode')
    # print(self.dataset)
  
  def mse(self,list1,list2):
    total_size=len(list1)+len(list2)
    mean_of_list1=np.mean(list1)
    mean_of_list2=np.mean(list2)
    list1=list1-mean_of_list1
    list2=list2-mean_of_list2
    list1=np.square(list1)
    list2=np.square(list2)
    mse_of_list1=np.sum(list1)*len(list1)/total_size
    mse_of_list2=np.sum(list2)*len(list2)/total_size
    return mse_of_list1+mse_of_list2

  def split(self,dataset):
    minmse=math.inf
    attribute_selected=0
    splitvalue=0
    # print('split function---')
    for i in range(np.size(dataset,1)-1):
      if(i not in self.columnx):
        unique_entries=np.unique(dataset[:,i])
        # print(unique_entries)
        # unique_entry_type=type(unique_entries[0])
        # print(unique_entry_type)
        if(isinstance(unique_entries[0],int) or isinstance(unique_entries[0],float)):
          mean_of_adjacent_unique_entries=[]
          for x in range(len(unique_entries)-1):
            mean_of_adjacent_unique_entries.append((unique_entries[x]+unique_entries[x+1])/2)
          # print(mean_of_adjacent_unique_entries)
          for item in mean_of_adjacent_unique_entries:
            list1=[]
            list2=[]
            for j in range(len(dataset[:,i])):
              # print(dataset[j][i])
              if(dataset[j][i]<=item):
                list1.append(dataset[j][-1])
              else:
                list2.append(dataset[j][-1])
            # print(len(list1))
            # print(len(list2))
            value_from_mse=self.mse(list1,list2)
            if(value_from_mse<minmse):
              minmse=value_from_mse
              attribute_selected=i
              splitvalue=item
          # print(minmse,attribute_selected,splitvalue)

        else:
          # print('str')
          # minmse=math.inf
          # attribute_selected=0
          # splitvalue=''
          for item in unique_entries:
            list1=[]
            list2=[]
            l1=[]
            l2=[]
            # print(item)
            for j in range(len(dataset[:,i])):
              if(dataset[j][i]==item):
                # l1.append(dataset[j][i])
                list1.append(dataset[j][-1])
              else:
                # l2.append(dataset[j][i])
                list2.append(dataset[j][-1])
            value_from_mse=self.mse(list1,list2)
            # print(value_from_mse)
            if(value_from_mse<minmse):
              minmse=value_from_mse
              attribute_selected=i
              splitvalue=item
            # print(list1)
            # print('---')
            # print(list2)
          # print(minmse,attribute_selected,splitvalue)
    self.columnx.append(attribute_selected)
    return minmse,attribute_selected,splitvalue
    # print(dataset[0,attribute_selected])

  def bintree(self,dataset):
    # if(np.size(dataset,0)==0):
    #   return None
    if(np.size(dataset,0)<=40):
      # print(dataset[:,-1])
      # print("size : ",np.size(dataset,0))
      val=np.mean(dataset[:,-1],axis=0)
      root=Node(-1,val)
      # root.leaf=True
      return root
    else:
      leftdataset=[]
      rightdataset=[]
      rightdataset=[]
      mse,column_idx,value=self.split(dataset)
      # print(mse,column_idx,value)
      root=Node(column_idx,value)
      #Split dataset into 2 subsections for left & right subtrees
      # print(type(value))
      if(isinstance(value,str)):
        for j in range(len(dataset[:,0])):
          if(dataset[j][column_idx]==value):
            # leftdataset=np.vstack([leftdataset,dataset[j,:]])
            leftdataset.append(dataset[j,:])
          else:
            # rightdataset=np.vstack([rightdataset,dataset[j,:]])
            rightdataset.append(dataset[j,:])
      else:
        for j in range(len(dataset[:,0])):
          if(dataset[j][column_idx]<=value):
            # leftdataset=np.vstack([leftdataset,dataset[j,:]])
            leftdataset.append(dataset[j,:])
          else:
            # rightdataset=np.vstack([rightdataset,dataset[j,:]])
            rightdataset.append(dataset[j,:])

      # print(np.size(leftdataset,0))
      # print(np.size(rightdataset,0))
      # print(rightdataset)
      if(np.size(leftdataset,0)==0):
        root.left=None
        newn=Node(-1,np.mean(np.array(rightdataset)[:,-1],axis=0))
        root.right=newn
        return root
      elif(np.size(rightdataset,0)==0):
        root.right=None
        newn=Node(-1,np.mean(np.array(leftdataset)[:,-1],axis=0))
        root.left=newn
      else:
        root.left=self.bintree(np.array(leftdataset))
        root.right=self.bintree(np.array(rightdataset))
      return root

  def inorder(self,root):
    if(root==None):
      return None
    self.inorder(root.left)
    print(root.val)
    self.inorder(root.right)

  def predictvalue(self,row,root):
    # print(root.attribute)
    # print(root.value)
    # print('----')
    if(root.attribute==-1):
      return root.value
    # print(row[root.attribute])
    if(isinstance(row[root.attribute],str)):
      if(row[root.attribute]==root.value):
        return self.predictvalue(row,root.left)
      else:
        return self.predictvalue(row,root.right)

    else:
      if(row[root.attribute]<=root.value):
        return self.predictvalue(row,root.left)
      else:
        return self.predictvalue(row,root.right)


  def predict(self,filename,root):
    pv=[]
    testdata=pd.read_csv(filename)
    testdata=testdata.drop(testdata.columns[0],axis=1)
    testdata=testdata.drop(testdata.columns[self.dropcolumn],axis=1)
    testdata['MSZoning']=testdata['MSZoning'].replace('C (all)','C')
    testdata['MasVnrType']=testdata['MasVnrType'].replace('None',np.nan)
    testdata.fillna(testdata.mean(),inplace=True)
    testdata.fillna(testdata.mode().iloc[0],inplace=True)
    self.testdf=testdata
    testdata.to_numpy()
    # print(row)
    # row=testdata.iloc[427,:]
    for i in range(np.size(testdata,0)):
      # print("i",i)

      row=testdata.iloc[i,:]
      pv.append(self.predictvalue(row,root))
    return pv

  def sklearn_decisiontree(self,testlabel):

    label_encoder = preprocessing.LabelEncoder()
    regr_1 = DecisionTreeRegressor(max_depth=2)
    regr_2 = DecisionTreeRegressor(max_depth=5)
    for col in self.dataset.columns:
      if(self.dataset[col].dtype=='object'):
        self.dataset[col]=label_encoder.fit_transform(self.dataset[col])
        self.testdf[col]=label_encoder.fit_transform(self.testdf[col])
    datanp=self.dataset.to_numpy()
    testnp=self.testdf.to_numpy()
    regr_1.fit(datanp[:,:-1], datanp[:,-1])
    regr_2.fit(datanp[:,:-1], datanp[:,-1])
    y_1 = regr_1.predict(testnp)
    y_2 = regr_2.predict(testnp)
    print('Sklearn Decision Tree Regressor')
    print('R2 score for depth 2 :',r2_score(testlabel,y_1))
    print('R2 score for depth 5 :',r2_score(testlabel,y_2))
    print('MSE (2): ',mean_squared_error(testlabel,y_1))
    print('MSE (5): ',mean_squared_error(testlabel,y_2))

  def mean_always(self,testlabel):
    print('Mean always approach')
    nparray=self.dataset.to_numpy()
    predicted_values=[]
    prices=nparray[:,-1]
    meanvalue=np.mean(prices)
    testnp=self.testdf.to_numpy()
    for i in range(np.size(testnp,0)):
      predicted_values.append(meanvalue)
    print('R2 Score:',r2_score(testlabel,predicted_values))
    print('MSE : ',mean_squared_error(testlabel,predicted_values))


Checking for count of null values in each column of the dataset and unique values in dataset

In [40]:
dt=DecisionTree()
filename='./Datasets/q3/train.csv'
dt.train(filename)
# print(dt.dataset)
datanumpy=dt.dataset.to_numpy()
# print(datanumpy[8,:])
# dt.split(datanumpy)
root=dt.bintree(datanumpy)
predictlabels=dt.predict('./Datasets/q3/test.csv',root)
testlabel=pd.read_csv('./Datasets/q3/test_labels.csv',header=None)
testlabel=testlabel.drop(testlabel.columns[0],axis=1)
print('R2 score : ',r2_score(testlabel.to_numpy(),predictlabels))
print('MSE : ',mean_squared_error(testlabel.to_numpy(),predictlabels))
print('MAE : ',mean_absolute_error(testlabel.to_numpy(),predictlabels))
dt.sklearn_decisiontree(testlabel.to_numpy())
dt.mean_always(testlabel.to_numpy())

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


R2 score :  0.6455370975250683
MSE :  1873267331.1057253
MAE :  28608.74709904811
Sklearn Decision Tree Regressor
R2 score for depth 2 : 0.5822160270362079
R2 score for depth 5 : 0.7106811223528589
MSE (2):  2207906843.1370697
MSE (5):  1528993860.8085887
Mean always approach
R2 Score: -0.007125647313121597
MSE :  5322462690.052036


# Observation
## Scratch decision tree

> As *leaf dataset count* increases r2 score increases, MSE decreases.

## Sklearn Decision tree
>As *depth* of tree increases, the MSE decreases.

## Mean Always approach
> Strict No! r2 score is negative i.e. worse than horizontal line  
> High MSE value